In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from camera import CameraReader, display_frame
from IPython.display import display
from numpy.fft import fft2
from numpy import r_
import cv2
import ipywidgets as widgets
from matplotlib.colors import LogNorm
import numpy as np
import sys
from array import array
from scipy import fftpack
import matplotlib.pyplot as plt
%matplotlib notebook
#matriz_para_huffman=np.zeros(256)

In [3]:
    # Implemente una función que limpie los frames
    # frame_limpio = eliminar_ruido(frame)
    # print(sys.getsizeof(frame_limpio))  #getsizeof retorna el tamaño en Bytes
    
    # Implemente una función que comprima los frames
    # frame_comprimido = comprimir(frame)
    # print(sys.getsizeof(frame_comprimido))
    
    # Implemente una función que descomprima los frames
    # frame = descomprimir(frame_compromido)
    # print(cam.error(frame))  # Error de compresión
    #display_frame(out, frame)

In [4]:
%matplotlib notebook

def eliminar_ruido(frame):
    spectrum = fftpack.fftshift(fftpack.fft2(frame))/255
    matriz_limpiadora = np.ones((480,848))
    delta=4
    matriz_limpiadora[260-delta:261+delta, 426:427] = 0
    matriz_limpiadora[220-delta:221+delta, 422:423] = 0
    retorno = ((255*(fftpack.ifft2(fftpack.ifftshift(spectrum * matriz_limpiadora)))))
    
    #plt.figure(figsize=(9, 5))
    #plt.imshow((np.abs(spectrum))* matriz_limpiadora, norm=LogNorm(vmin=20))
    

    
    return retorno.astype(np.uint8)


def dct2(a):
    return fftpack.dct( fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return fftpack.idct( fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')


def comprimir(frame):
    global imsize
    imsize = frame.shape
    dct = np.zeros(imsize)
    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            dct[i:(i+8),j:(j+8)] = dct2( frame[i:(i+8),j:(j+8)] )
    pos = 128
    thresh = 0.012
    dct_thresh = dct * (abs(dct) > (thresh*np.max(dct)))
    percent_nonzeros = np.sum( dct_thresh != 0.0 ) / (imsize[0]*imsize[1]*1.0)
    return dct_thresh

def antitransformada(dct_thresh):
    global imsize
    im_dct = np.zeros(imsize)
    
    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            im_dct[i:(i+8),j:(j+8)] = idct2( dct_thresh[i:(i+8),j:(j+8)] ).astype(np.uint8)
            
    return im_dct.astype(np.uint8)

def llenado_de_tabla_de_pixeles(frame):
    global matriz_para_huffman
    for i in range(480):
        for j in range(848):
            matriz_para_huffman[0][frame[i][j]]=matriz_para_huffman[0][frame[i][j]]+1
            matriz_para_huffman[1][frame[i][j]]=frame[i][j]

def bubbleSort(alist):
    for passnum in range(len(alist[0])-1,0,-1):
        for i in range(passnum):
            if alist[0][i]<alist[0][i+1]:
                temp = alist[0][i]
                temp2 = alist[1][i]
                alist[0][i] = alist[0][i+1]
                alist[1][i] = alist[1][i+1]
                
                alist[0][i+1] = temp
                alist[1][i+1] = temp2
                
                
def shannon_method(len_lista, i, end, n):
    global arreglo_de_shannon, lista, largo_datos_relevantes, contador
    j = i
    contador += 1
    suma=0.0
    if(len_lista >2):
        while suma<(n/2) and i < end:
            suma += lista[i]
            i += 1
        if (i == end):
            suma2 = suma
            suma = 0.0
            i = j
            while(suma<(suma2/2)):
                suma += lista[i]
                i += 1
        for t in range(j , i):
            arreglo_de_shannon[t] += "0"
        if(contador < largo_datos_relevantes+50):
            shannon_method(i-j, j, i, n/2)
        
        for t in range(i , end):
            arreglo_de_shannon[t] += "1"
        if(contador < largo_datos_relevantes+50):
            shannon_method(end-i, i, end, n/2)

    if(len_lista == 2):
        arreglo_de_shannon[i] += "0"
        arreglo_de_shannon[i+1] += "1"
    if(len_lista == 1):
        arreglo_de_shannon[i] += "0"
    mat = ["" for i in range(255)]
    for i in range(255):
        for j in range(datos_relevantes(matriz_suplente_para_huffman[0])):
            if(matriz_probabilidades[1][j] == i):
                mat[i] = arreglo_de_shannon[i]
    return(mat)
        
def datos_relevantes(datos):
    count=0
    for i in range(len(datos)):
        if(datos[i] != 0):
            count+=1
    return count


cam = CameraReader()
vid_height, vid_width = cam.get_resolution()
print("Resolución del video: {0}x{1}p".format(vid_width, vid_height))
print("Cuadros por segundo: {0}".format(cam.get_fps()))

out = widgets.Image(layout=widgets.Layout(width=str(vid_width), 
                                          height=str(vid_height)))

matriz_para_huffman = [[0 for i in range(256)] for j in range(2)]
display(out)
imsize = 0


suma=0
count=0
delta_escalado=8


for frame in cam:
    frame = eliminar_ruido(frame)
    
    frame = (comprimir(frame)//delta_escalado)*delta_escalado
    frame = antitransformada(frame)
    llenado_de_tabla_de_pixeles(frame)
    
    
    display_frame(out, frame)


Iniciando lector de camara
OpenCV version: major: 4, minor: 1, subminor: 0
Resolución del video: 848x480p
Cuadros por segundo: 29.97002997002997


Image(value=b'', layout="Layout(height='480', width='848')")

/usr/local/lib/python3.6/dist-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  app.launch_new_instance()


In [5]:
for i in range(256):
    print(matriz_para_huffman[0][i],matriz_para_huffman[1][i])

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 45
2 46
0 0
1 48
2 49
3 50
6 51
5 52
20 53
22 54
31 55
53 56
70 57
107 58
180 59
302 60
455 61
789 62
1703 63
4007 64
8345 65
15093 66
35392 67
92896 68
236695 69
635903 70
1403164 71
2110759 72
2990435 73
4090179 74
4894631 75
5619087 76
6819281 77
7749775 78
8219328 79
8506342 80
8808234 81
8922790 82
8979306 83
9139479 84
9095296 85
8834515 86
8432418 87
8143274 88
7912814 89
7804101 90
7792761 91
7680048 92
7565133 93
7473456 94
7446704 95
7390667 96
7173364 97
7084264 98
6895122 99
6602863 100
6275893 101
6074055 102
5782146 103
5412080 104
5186869 105
4982363 106
4606245 107
4270281 108
4009547 109
3712742 110
3431303 111
3193150 112
3012320 113
2843362 114
2675929 115
2539146 116
2411994 117
2264516 118
2136168 119
2024024 120
1893875 121
1747891 122
1635634 123
1514301 124
1412915 125
1314182 126
12

In [6]:
matriz_suplente_para_huffman = matriz_para_huffman
suma=0
for i in matriz_suplente_para_huffman[0]:
    suma = suma + i
print(suma)

366336000


In [7]:

largo_datos_relevantes = datos_relevantes(matriz_suplente_para_huffman[0])
largo_datos_relevantes = 255
bubbleSort(matriz_suplente_para_huffman)

matriz_suplente_para_huffman = np.array(matriz_suplente_para_huffman) 
matriz_probabilidades = np.zeros((2,largo_datos_relevantes))
    
    
total = 848*480*900
suma_probabilidades = 0.0

for i in range(largo_datos_relevantes):
    matriz_probabilidades[0][i] = (matriz_suplente_para_huffman[0][i])/total
    matriz_probabilidades[1][i] = (matriz_suplente_para_huffman[1][i])
    suma_probabilidades += matriz_probabilidades[0][i]
for i in range(largo_datos_relevantes):
    print(matriz_probabilidades[0][i], matriz_probabilidades[1][i])



0.024948350694444444 84.0
0.0248277428371768 85.0
0.024511120938155138 83.0
0.024356847265897975 82.0
0.024115879957197765 86.0
0.024044139806079664 81.0
0.023220054813067784 80.0
0.023018261923480082 87.0
0.02243658280922432 79.0
0.022228975585255065 88.0
0.02159988098357792 89.0
0.02130312336215933 90.0
0.021272168173480085 91.0
0.021154827808350804 78.0
0.020964491614255766 92.0
0.020650804179769392 93.0
0.02040055031446541 94.0
0.020327524458420686 95.0
0.02017455832896576 96.0
0.019581378843466106 97.0
0.019338159503843468 98.0
0.01882185207023061 99.0
0.01861482627969951 77.0
0.018024062609189376 100.0
0.01713152133560447 101.0
0.016580557193396225 102.0
0.015783723139412998 103.0
0.015338615369496855 76.0
0.014773541229909154 104.0
0.014158775004367576 105.0
0.013600527930642908 106.0
0.013361042867749825 75.0
0.012573825668238994 107.0
0.011656733162997904 108.0
0.011165102528825997 74.0
0.010944998580538086 109.0
0.010134799746680643 110.0
0.009366546012403913 111.0
0.00871645

In [8]:
arreglo_de_shannon = ["" for i in range(255)]
lista = matriz_probabilidades[0]
suma = 0.0
contador = 0

mat = shannon_method(largo_datos_relevantes, 0,largo_datos_relevantes, 1.0)
for i in range(255):
    print(arreglo_de_shannon[i] , matriz_probabilidades[1][i])
    





000000 84.0
000001 85.0
000010 83.0
000100 82.0
000101 86.0
000110 81.0
001000 80.0
001001 87.0
001010 79.0
00110 88.0
00111 89.0
010000 90.0
010001 91.0
010010 78.0
010100 92.0
010101 93.0
010110 94.0
011000 95.0
011001 96.0
011010 97.0
011011 98.0
01110 99.0
01111 77.0
100000 100.0
100001 101.0
100010 102.0
100011 103.0
1001000 76.0
1001001 104.0
1001010 105.0
100110 106.0
1010000 75.0
1010001 107.0
1010010 108.0
1010100 74.0
1010101 109.0
1010110 110.0
1011000 111.0
1011001 112.0
1011010 113.0
1011011 73.0
1011100 114.0
1011101 115.0
1011110 116.0
11000000 117.0
11000001 118.0
11000010 119.0
11000100 72.0
11000101 120.0
11000110 121.0
11001000 122.0
11001001 123.0
11001010 124.0
11001011 125.0
11001100 159.0
11001101 71.0
11001110 160.0
110100000 163.0
110100001 164.0
110100010 161.0
11010010 158.0
11010011 165.0
110101000 162.0
110101001 126.0
110101010 166.0
11010110 157.0
110110000 127.0
110110001 167.0
110110010 156.0
11011010 128.0
11011011 168.0
110111000 129.0
110111001 155.0

In [9]:

mat = ["" for i in range(255)]
for i in range(255):
    for j in range(255):
        if(matriz_probabilidades[1][j] == i and matriz_probabilidades[1][j] != 0):
            if(i==84):
                print(matriz_probabilidades[1][j], arreglo_de_shannon[j])
            mat[i] = arreglo_de_shannon[j]
print(mat, matriz_probabilidades[1][0])

84.0 000000
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '111111111111111111111100', '11111111111111111111100', '', '111111111111111111111101', '11111111111111111111101', '1111111111111111111101', '1111111111111111111010', '1111111111111111111100', '111111111111111111001', '111111111111111111000', '11111111111111111010', '1111111111111111010', '1111111111111111001', '1111111111111111000', '111111111111111001', '11111111111111010', '11111111111111000', '1111111111111001', '111111111111010', '11111111111010', '11111111111000', '1111111111001', '11111111101', '11111111000', '111111011', '111110110', '11001101', '11000100', '1011011', '1010100', '1010000', '1001000', '01111', '010010', '001010', '001000', '000110', '000100', '000010', '000000', '000001', '000101', '001001', '00110', '00111', '010000', '010001', '010100', '010101', '010110', '011000', '0110

In [10]:
def codificacion_de_frame(frame):
    global mat
    cadena = ""
    for i in range(848):
        for j in range(480):
            cadena += mat[frame[j][i]]
    return cadena

def decodificacion_de_frame(cadena):
    global mat
    inicio = 0
    final = 0
    cont = -1
    frame = [0 for i in range(255)]
    for i in range(848):
        for j in range(480):
            while(cadena[inicio:final]!=arreglo_de_shannon[cont]):
                cont+=1
                cadena[inicio:final]
                
                
    return cadena

SyntaxError: invalid syntax (<ipython-input-10-34aa12a61a6c>, line 14)

In [ ]:
cam = CameraReader()

matriz_para_huffman = [[0 for i in range(256)] for j in range(2)]
display(out)
imsize = 0


suma=0
count=0
delta_escalado=64
cadena_de_bits = ""
print(len(matriz_para_huffman[0]))




"""
for frame in cam:
    frame = eliminar_ruido(frame)
    print("largo frame: ",os.stat(frame).st_size)
    frame = (comprimir(frame)//delta_escalado)*delta_escalado
    frame = antitransformada(frame)
    #llenado_de_tabla_de_pixeles(frame)
    frame_codificado = codificacion_de_frame(frame)
    
    #frame_decodificado = decodificacion_de_frame(frame)
    display_frame(out, frame)
"""
cam = CameraReader()

next(cam)
frame = eliminar_ruido(frame)
frame = (comprimir(frame)//delta_escalado)*delta_escalado
frame = antitransformada(frame)
frame_codificado = codificacion_de_frame(frame)
print("frame_codificado: ", len(frame_codificado))
display_frame(out, frame)

In [ ]:
print(len(frame_codificado))


In [12]:
cadena = "hola"
cadena[0:2]

'ho'